# Xarray-simlab: customize models

Xarray-simlab is a modular framework. The motivation behind this framework is to enable an interactive user experience for *both* model exploration (inspection, run) *and* model customization. 

We'll see here how we can customize xarray-simlab models by adding, dropping or replacing some processes (with other available processes).

In [ ]:
import xsimlab as xs

## Fastscape basic model

As an example, let's first import a model from fastscape

In [ ]:
from fastscape.models import basic_model

This basic model simulates the evolution of the topographic surface under the action of tectonic processes (vertical uplift) and erosion processes (bedrock channel SPL, hillslope diffusion).

Let's visualize the model

In [ ]:
basic_model.visualize(show_inputs=True)

SPL's drainage area is calculated by computing convergent flow paths (single flow direction). The `SingleFlowRouter` process is used:

In [ ]:
basic_model.flow

## Yes but I want divergent flow paths

Fortunately, fastscape provides a process for that: `MultipleFlowRouter`

In [ ]:
from fastscape.processes import MultipleFlowRouter

To use it instead of `SingleFlowRouter`, just update the processes:

In [ ]:
mfd_model = basic_model.update_processes({'flow': MultipleFlowRouter})

`mdf_model` is a new xarray-simlab model, built from `basic_model`.

When xarray-simlab builds a new model, it re-scans the variables in the processes in order to re-build from scratch the graph of processes (+ sorting those processes) and determine the model input variables (fortunately, the whole operation is fast).

If we compare `MultipleFlowRouter` with `SingleFlowRouter`, we can see that the former has an additional variable: the flow partition slope exponent (`slope_exp`):

In [ ]:
mfd_model.flow

This is an input parameters, that is a new model input (we can see it in the graph below, while it is absent in the graph above):

In [ ]:
mfd_model.visualize(show_inputs=True)

## Ok, but I'd like to simulate the erosion of an escarpement (no uplift), with channel transport/deposition and with differential erosion (rock vs. sediment layer)

Right, fastscape provides more processes. Note that it is also possible to simply drop processes.

In [ ]:
from fastscape.processes import (
    Bedrock,
    BareRockSurface,
    DifferentialLinearDiffusion,
    DifferentialStreamPowerChannelTD,
    Escarpment,
    UniformSedimentLayer,
)


escarpment_model = (
    mfd_model
    .drop_processes('uplift')
    .update_processes({
        'init_topography': Escarpment,
        'bedrock': Bedrock,
        'active_layer': UniformSedimentLayer,
        'init_bedrock': BareRockSurface,
        'flow': MultipleFlowRouter,
        'spl': DifferentialStreamPowerChannelTD,
        'diffusion': DifferentialLinearDiffusion
    })
)

In [ ]:
escarpment_model.visualize(show_inputs=True)

## Orographic precipitation, please?

Alright, the fastscape package itself doesn't provide any built-in process for that. However, nothing prevents implementing extensions in thrid-party packages, like the `orographic_precipitation` package developped by Raphael Lange (https://github.com/fastscape-lem/orographic-precipitation).

In [ ]:
from orographic_precipitation.fastscape_ext import (
    OrographicPrecipitation,
    OrographicDrainageDischarge
)

There's a process for computing the topographic-dependent precipitation rate

In [ ]:
xs.process_info(OrographicPrecipitation)

And another process to compute water discharge (that would replace drainage area)

In [ ]:
xs.process_info(OrographicDrainageDischarge)

Let's include this in the escarpment model

In [ ]:
precip_model = escarpment_model.update_processes({'precip': OrographicPrecipitation,
                                                  'drainage': OrographicDrainageDischarge})

In [ ]:
precip_model.visualize(show_inputs=True)

## What about rock layers?

Sorry, to my knowledge there's no fastscape-compatible process for that yet. You'll need to write your own process (we'll see that later). We hope that in the future more and more researchers will share their processes (for example, a topographic driven life-evolution model extension should be available soon).

## Fastscape model presets

The fastscape Python package provides a few built-in models, which are built on top of each other, like shown here above.

The core model object is the `bootstrap_model`: it doesn't implement any physical process but instead provides the skeleton of a fastscape model (grid, boundary conditions, topography, etc.):

In [ ]:
from fastscape.models import bootstrap_model

bootstrap_model.visualize(show_inputs=True)

`basic_model` is built directly on top of that `bootstrap_model`.

At the other end of the chain, the `marine_model` is the most complex one:

In [ ]:
from fastscape.models import marine_model

marine_model.visualize(show_inputs=True)